In [ ]:
#v10


In [ ]:
from IPython.display import HTML
from vevestaX import vevesta as v
from IPython.display import display, HTML
from PIL import Image
from dask.distributed import Client
from dask.distributed import as_completed
from datetime import datetime
from dateutil.relativedelta import relativedelta
from matplotlib.dates import DateFormatter
from pandas.io.formats.style import Styler
from pivottablejs import pivot_ui
from pmdarima.arima import auto_arima
from scipy import stats
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import TimeSeriesSplit
import re
from statsmodels.stats.outliers_influence import summary_table
import datetime as dt
import matplotlib
import matplotlib as mp
import matplotlib as mpl
import matplotlib.dates as md
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pandas_market_calendars as mcal
import pickle
import rpy2
import seaborn as sns
import sqlite3
import statsmodels.api as sm
import warnings
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import statsmodels
from statsmodels.tsa.exponential_smoothing.ets import ETSModel
from rpy2.robjects import pandas2ri
from rpy2.robjects.conversion import localconverter
from rpy2.robjects.packages import importr
import rpy2
import rpy2.robjects as ro
import time
from clustergram import Clustergram
from sklearn.cluster import DBSCAN
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from scipy.cluster.vq import vq
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.preprocessing import scale
from k_means_constrained import KMeansConstrained
from scipy.stats import f
from matplotlib.colors import LinearSegmentedColormap
from IPython.display import HTML as html_print
from IPython.display import display


pd.options.display.max_columns = 50
pd.options.display.max_rows = 200

import pandas as pd
from rpy2.robjects import pandas2ri
from prophet import Prophet

In [ ]:
#if not sys.warnoptions:
#    import warnings
#    warnings.simplefilter("ignore")

wd = os.getcwd()

if (os.defpath==".;C:\\bin"):
    os.environ['R_HOME'] = 'C:/Users/User/Documents/R/R-4.1.2'
    os.environ['R_LIBS'] = 'C:/Users/User/Documents/R/R-4.1.2/library'
    from OLS_LR_DiagnosticPlots.ModelDiagnostics import Plot
else:
    os.environ['R_HOME'] = '/mnt/distvol/R/4.0.5/lib64/R/'

pandas2ri.activate()


In [ ]:

#robjects.ro['version']

base = importr('base')
#grdevices = importr('grDevices')
print(base._libPaths())

timetk = importr('timetk')
magrittr = importr('magrittr')
dplyr = importr('dplyr')
tidyverse = importr('tidyverse')
nbclust = importr('NbClust')
grdevices = importr('grDevices')


In [ ]:
si = ['03-31','06-30','09-30','12-31']
               
[start,end,prices_df, sp1500_index_df, sp500, sp600, sp400, market_data, completed_fred_pvt, completed_bonds, completed_bonds_pvt, sectors, indexes, screener_sorted, dict_sectors, dict_indexes, dict_sectors_reverse, invert_dict_indexes, list_sector_n_indexes, list_stocks] = pickle.load(open('./data_object.pkl', 'rb'))

fred_friendly_names = ['Consumer Loans','Copper','Iron and Steel','Gold','Unemployment','Market volatility','Commercial-Industrial Loans','Average Weekly Hours','Credit to Income','Consumer Confidence','Oil Prices','Inflation','Housing Prices','Interest Rates','10 Year to 3 Month','Recession Indicator','OECD Leading Indicator','Coincident Index','Index: Industrial Production','Mfr Orders Excl AC','Personal Expenditures',]
fred_names = ['CONSUMER','WPUSI019011','WPU101','GVZCLS','UNRATE','VIXCLS','BUSLOANS','AWHAETP','UMCSENT','TDSP','DCOILWTICO','CPIAUCSL','CSUSHPINSA','FEDFUNDS','T10Y3M','USREC','USALOLITONOSTSAM','USPHCI','INDPRO','NEWORDER','PCE']

dict_fred = dict(zip(*[fred_names,fred_friendly_names]))

nyse = mcal.get_calendar('NYSE')
nyse_dates = nyse.schedule(start_date=start, end_date=end)

fundamentals_quarterlies = pickle.load(open('./fundamental.pkl', 'rb'))
fundamental_entries = [e[0] for e in fundamentals_quarterlies]

sectors.columns = ['Symbol']
indexes.columns = ['Symbol']

newDates = pd.date_range((end+ dt.timedelta(7)).strftime('%Y-%m-%d'), (end + dt.timedelta(92)).strftime('%Y-%m-%d'), freq='W-'+nyse_dates.index[-1].strftime('%a')).map(lambda t: t.strftime('%Y-%m-%d'))

db_filename = 'todo.db'

db_is_new = not os.path.exists(db_filename)

conn = sqlite3.connect(db_filename)

if db_is_new:
    print('Need to create schema')
else:
    print('Database exists, assume schema does, too.')

conn.close()

In [ ]:
#completed_fred_pvt_df = completed_fred_pvt.asfreq('D').reindex(nyse_dates.index).interpolate(method='time')
completed_fred_pvt_df = completed_fred_pvt

temp_new = completed_fred_pvt.asfreq('D').interpolate(method='time',limit_area='inside').reindex(nyse_dates.index)
temp_dates = pd.date_range(completed_fred_pvt.index[0].strftime('%Y-%m-%d'), completed_fred_pvt.index[-1].strftime('%Y-%m-%d'), freq='D').map(lambda t: t.strftime('%Y-%m-%d'))
nyse_inbetween_temp_dates = nyse_dates.index[(nyse_dates.index >= temp_dates[0]) & (nyse_dates.index <= temp_dates[-1])]
temp_new = temp_new.reindex(nyse_inbetween_temp_dates)

completed_fred_pvt_df = temp_new
completed_fred_pvt_df.columns = [name[1] for name in completed_fred_pvt_df.columns]

np.array(completed_fred_pvt_df.index.map(lambda t: t.strftime('%Y-%m-%d')))[np.arange(0,len(completed_fred_pvt_df.index),int(len(completed_fred_pvt_df.index)/10))]

In [ ]:
n_ahead = 13

cv_inner = TimeSeriesSplit(n_splits=10,test_size=n_ahead)
cv_outer = TimeSeriesSplit(n_splits=11,test_size=n_ahead)


In [ ]:

def cstr(s, color='black'):
    return "<text style=color:{}>{}</text>".format(color, s)

def print_color(t):
    print(t)
    display(html_print(' '.join([cstr(ti, color=ci) for ti,ci in t])))


In [ ]:
def findknee(xdata):
    rate_of_change=(xdata[0]-xdata[-1])/(len(xdata)-1)
    #print(rate_of_change)
    delta = xdata-xdata[-1]
    deltas = []
    deltas.append(delta[0])
    for d in range(1,len(xdata)):
        deltas.append(deltas[d-1]-rate_of_change)
    #print(deltas)
    for d in range(0,len(xdata)):
        deltas[d]=delta[d]-deltas[d]
    return(np.round(np.abs(deltas)))

def deriveANOVA(clf, df):

    labels = clf.labels_
    clusters = clf.n_clusters
    centers = clf.cluster_centers_
        
    within_ss = []

    for n in range(0,clusters):
      #WSS means the sum of distances between the points and the corresponding centroids for each cluster
      data = df[labels==(n)]
      within_ss.append(((data - centers[n])**2).sum(1).sum())

    WSS = total_within_ss = np.sum(within_ss)

    print('wss',total_within_ss)

    #sum of ((deviation from variable means) squared)
    tot_ss = np.sum(np.sum((df-df.mean())**2))
    print('tot_ss',tot_ss)

    cluster_BSS = []
    for n in range(0,clusters):
      #sum((variable/column means cluster - variable/column means data)^2)*len(cluster members)
      BSS = np.sum((df[labels==(n)].mean()-np.array(np.mean(df)))**2)*len(df[labels==(n)])
      cluster_BSS.append(BSS)

    BSS = np.sum(cluster_BSS)
    print('bss',BSS)
    
    return(tot_ss, BSS, within_ss)

def findOptimalK_ANOVA(df, mink=2, maxk=6, init_min=2, init_max=5):

    tss = []
    bss = []
    wss = []

    for k in range(mink,maxk):
        print(k)

        print(max(init_min,k+1))
        size_min_ = max(init_min,k+1)
        size_max_ = max(np.ceil(len(df)/k),init_max)
        if(size_min_>size_max_):
            break
        else:
            clf = KMeansConstrained(n_clusters=k, size_min=size_min_, size_max=size_max_, init='k-means++', n_init=100, max_iter=100, tol=0.0001, verbose=False, random_state=int(time.time()), copy_x=True, n_jobs=4)
            clf.fit_predict(np.array(df))

            tot_ss, BSS, within_ss = deriveANOVA(clf, df)

            wss.append(within_ss)
            tss.append(tot_ss)
            bss.append(BSS)

    return(tss, bss, wss)

In [ ]:
def time_to_int(dateobj):
    total = int(dateobj.strftime('%S'))
    total += int(dateobj.strftime('%M')) * 60
    total += int(dateobj.strftime('%H')) * 60 * 60
    total += (int(dateobj.strftime('%j')) - 1) * 60 * 60 * 24
    total += (int(dateobj.strftime('%Y')) - 1970) * 60 * 60 * 24 * 365
    return total


In [ ]:
def highlight_cells(val, color_if_true, color_if_false, threshold_):
    color = color_if_true if val >= threshold_ else color_if_false
    return 'background-color: {}'.format(color)

In [ ]:
def derive_fb_test_error(npa):
    n_ahead = 13

    subset_train = npa[0]
    subset_test = npa[1]

    base_model = Prophet()

    prophet_df_base = subset_train['Adj Close'].reset_index()#data.reset_index()
    prophet_df_base.columns = ['ds','y']
    prophet_df_base['y'] = np.log(prophet_df_base['y'])

    base_model.fit(prophet_df_base)

    future_base = base_model.make_future_dataframe(periods = n_ahead,freq='W-'+subset_train.index[-1].strftime('%a'))

    forecast_base = base_model.predict(future_base).set_index('ds')[['yhat','yhat_lower','yhat_upper']]
    df_pred_base = np.exp(forecast_base)

    rmse_ = mean_squared_error(subset_test['Adj Close'].tail(n_ahead), df_pred_base['yhat'].tail(n_ahead), squared=True)
    return([[subset_test['Adj Close'].tail(n_ahead)],[df_pred_base['yhat'].tail(n_ahead)],rmse_])

In [ ]:
"""
symbol_ = 'BA'

if(str(symbol_)=='None'):
    run=False
else:
    run=True

if(run):
    client = Client('192.168.3.100:8786')
    #client = Client(n_workers=4,threads_per_worker=1)

    s = symbol_
    msize = 2
    date_form = DateFormatter("%Y-%m-%d")

    metrics_df_ = pd.DataFrame(pd.DataFrame(screener_sorted.loc[s]).T[['volume_factor','Adj Close','adf','hurst']])
    subset=prices_df[prices_df['Symbol']==s]
    
    y = subset['Adj Close']
    data = subset[['Adj Close']].asfreq('D').interpolate().asfreq('W-'+subset.index[-1].strftime('%a'))
"""

In [123]:
def plot_(symbols):    

    #symbols = ['GO']
    decision_metrics_df = pd.DataFrame()

    for symbol_ in symbols:

        print(symbol_)
        if(str(symbol_)=='None'):
            run=False
        else:
            run=True

        if(run):
            client = Client('192.168.3.100:8786')
            #client = Client(n_workers=4,threads_per_worker=1)

            s = symbol_
            msize = 2
            date_form = DateFormatter("%Y-%m-%d")

            metrics_df_ = pd.DataFrame(pd.DataFrame(screener_sorted.loc[s]).T)#[['volume_factor','Adj Close','adf','hurst']])
            subset=prices_df[prices_df['Symbol']==s]

            dates = []
            for t in subset.index.values:
                d = pd.Timestamp(t).strftime('%Y-%m-%d')
                dates.append(d)
                dto = datetime.strptime(d, '%Y-%m-%d').date()

            old_ordinal = [datetime.strptime(i, '%Y-%m-%d').toordinal() for i in dates]
            new_ordinal = old_ordinal + md.date2num(np.datetime64('0000-12-31'))
            x = new_ordinal    

            lookup_index_ = []

            labels = list()
            print("symbol:",s)

            symbol_sector = screener_sorted.loc[symbol_]['Sector Symbol']

            print("sector:",dict_sectors_reverse[symbol_sector])
            symbol_index = screener_sorted.loc[s]['Index Symbol']

            try:
                matched_index_name = indexes.iloc[np.where(indexes['Symbol'].values==s)].index[0]
            except:
                try:
                    matched_index_name = screener_sorted.loc[s]['Index Symbol']
                except:
                    matched_index_name = "error"

            print("Index:",dict_indexes[matched_index_name])

            print("Volume Factor:",screener_sorted.iloc[np.where(screener_sorted.index==s)]['volume_factor'][0])

            print("sector_risk_trend_factor:",screener_sorted[screener_sorted.index==s]['sector_risk_trend_factor'][0])
            print("risk trend factor:",screener_sorted.iloc[np.where(screener_sorted.index==s)]['risk_trend_factor'][0])

            temp = pd.DataFrame(market_data.loc[s])
            print(temp[np.array(temp!='error')].replace([np.inf,'inf','error', -np.inf], np.nan).dropna().T)

            last_date = pd.DataFrame(subset.iloc[-1][['21dBOLD','21dMA-TP','21dBOLU']],index=[prices_df[prices_df['Symbol']==s].index[-1].strftime('%Y-%m-%d')]).index[0]

            bbands = pd.DataFrame(subset.iloc[-1][['Adj Close','21dBOLD','21dMA-TP','21dBOLU','90dSMA','90dSDev']]).T
            bbands['90d_lower'] = bbands['90dSMA']-bbands['90dSDev']*2
            bbands['90d_upper'] = bbands['90dSMA']+bbands['90dSDev']*2

            bbands.index = [last_date]
            display(bbands)

            display(metrics_df_)

            mean_revert_flag = False

            if(metrics_df_['adf'][0]<.05):
                print("ADFuller H0 rejected @ .05, Mean Reverting TS")
                if(bbands['Adj Close'][0]<bbands['90d_lower'][0]):
                    colored_text = colored(255, 0, 0, "Quarter Mean Reverting buy signal")
                    print(colored_text)
                    mean_revert_flag = True
                if(bbands['Adj Close'][0]<bbands['21dBOLU'][0]):
                    colored_text = colored(255, 0, 0, "Month Mean Reverting buy signal")
                    print(colored_text)
                    mean_revert_flag = True    

            l_axis_legend = pd.DataFrame(['symbol', 'trailing_1yr_max', 'trailing_1yr_min', '30d_vol_2yr','risk_trend_factor/linear','supply_trend_1yr'],index=['blue','green','red','orange','black','yellow'],columns=['legend'])

            r_axis_legend = pd.DataFrame(['sector','index'],index=['magenta','cyan'],columns=['legend'])

            display_side_by_side([l_axis_legend, r_axis_legend], ['l axis', 'r axis'])        

            f, (ax1, ax2, ax3, ax4, ax5) = plt.subplots(1, 5, sharey=False,figsize=(22,5))
            ax1.plot(subset['30d_vol_2yr'],color='orange')
            #plt.title('30d_vol_2yr')
            ax1.set_xticklabels(subset[['30d_vol_2yr']].dropna().index, rotation = 45)

            locator = matplotlib.dates.AutoDateLocator()
            formatter = matplotlib.dates.ConciseDateFormatter(locator)

            ax1.xaxis.set_major_locator(locator)
            ax1.xaxis.set_major_formatter(formatter)

            ax1.xaxis.set_major_formatter(date_form)

            ax2.plot(subset['risk_trend_factor'],color='black')
            ax2.set_xticklabels(subset[['risk_trend_factor']].dropna().index, rotation = 45)

            locator = matplotlib.dates.AutoDateLocator()
            formatter = matplotlib.dates.ConciseDateFormatter(locator)

            ax2.xaxis.set_major_locator(locator)
            ax2.xaxis.set_major_formatter(formatter)

            ax2.xaxis.set_major_formatter(date_form)

            ax3_ = ax3.twinx()
            ax5_ = ax5.twinx()        
            ax4_ = ax4.twinx()

            y = subset['Adj Close']

            def myfunc(x):
                return slope * x + intercept

            slope, intercept, r, p, std_err = stats.linregress(x, y)

            mymodel = list(map(myfunc, x))

            ax3.plot(x, y,marker = '.',markersize=msize, color = 'b')
            ax3.plot(x, subset['trailing_1yr_max'],marker = '.',markersize=msize, color = 'g')
            ax3.plot(x, subset['trailing_1yr_min'],marker = '.',markersize=msize, color = 'r')
            ax3_.plot(x, subset['supply_trend_1yr'],marker = '.',markersize=msize, color = 'y')
            ax3.plot(x, mymodel, color = 'k')

            l = matplotlib.dates.AutoDateLocator()
            f = matplotlib.dates.ConciseDateFormatter(l)    

            ax3.set_xticklabels(ax3.get_xticks(), rotation = 45)

            locator = matplotlib.dates.AutoDateLocator()
            formatter = matplotlib.dates.ConciseDateFormatter(locator)

            ax3.xaxis.set_major_locator(locator)
            ax3.xaxis.set_major_formatter(formatter)
            ax3.xaxis.set_major_formatter(date_form)

            if(symbol_sector=='error'):
                pass
            else:
                if(np.sum(sectors['Symbol'].values==s)>0):
                    pass
                else:
                    sector_subset = prices_df[prices_df['Symbol']==symbol_sector]
                    print("Sector Volume Factor:",screener_sorted.iloc[np.where(screener_sorted.index==symbol_sector)]['volume_factor'][0])
                    print("Sector Risk Trend Factor:",screener_sorted.iloc[np.where(screener_sorted.index==symbol_sector)]['risk_trend_factor'][0])
                    #ax3_.plot(x, (sector_subset['Adj Close']).iloc[-len(subset):],marker = '.',markersize=msize,color='m',linestyle=(0, (3, 10, 1, 10)))
                    ax4.plot(x, (sector_subset['Adj Close']).iloc[-len(subset):],marker = '.',markersize=msize,color='m',linestyle=(0, (3, 10, 1, 10)))
                    ax4.plot(x, (sector_subset['trailing_1yr_max']).iloc[-len(subset):],marker = '.',markersize=msize, color = 'g')
                    ax4.plot(x, (sector_subset['trailing_1yr_min']).iloc[-len(subset):],marker = '.',markersize=msize, color = 'r')
                    ax4_.plot(x, (sector_subset['supply_trend_1yr']).iloc[-len(subset):],marker = '.',markersize=msize,color='y',linestyle=(0, (3, 10, 1, 10)))
                    ax4.set_xticklabels(subset.dropna().index, rotation = 45)
                    ax4.xaxis.set_major_formatter(date_form)

            if(str(screener_sorted.loc[s]['Index Symbol'])!='nan'):
                if(screener_sorted.loc[s]['Index Symbol']!='error'):
                    index_subset = prices_df[prices_df['Symbol']==screener_sorted.loc[s]['Index Symbol']]
                    print("Index Volume Factor:",screener_sorted.iloc[np.where(screener_sorted.index==screener_sorted.loc[s]['Index Symbol'])]['volume_factor'][0])
                    print("Index Risk Trend Factor:",screener_sorted.iloc[np.where(screener_sorted.index==screener_sorted.loc[s]['Index Symbol'])]['risk_trend_factor'][0])
                    ax5.plot(x, index_subset['Adj Close'].iloc[-len(subset):],marker = '.',markersize=msize,color='c',linestyle=(0, (1, 10)))
                    ax5.plot(x, index_subset['trailing_1yr_max'].iloc[-len(subset):],marker = '.',markersize=msize, color = 'g')
                    ax5.plot(x, index_subset['trailing_1yr_min'].iloc[-len(subset):],marker = '.',markersize=msize, color = 'r')
                    ax5_.plot(x, index_subset['supply_trend_1yr'].iloc[-len(subset):],marker = '.',markersize=msize,color='y',linestyle=(0, (1, 10)))
                    ax5.set_xticklabels(index_subset.dropna().index, rotation = 45)
                    ax5.xaxis.set_major_formatter(date_form)

            plt.show()

            data = subset[['Adj Close']].asfreq('D').interpolate().asfreq('W-'+subset.index[-1].strftime('%a'))

            npa_rmse_inner = []

            for trainv_ix, test_ix in cv_inner.split(data.index):

                npa_rmse_inner.append([data.iloc[trainv_ix],data.iloc[test_ix]])

            npa_rmse_outer = []

            for trainv_ix, test_ix in cv_outer.split(data.index):

                npa_rmse_outer.append([data.iloc[trainv_ix],data.iloc[test_ix]])        

            print("fbprophet next 13 weeks")

            #starting from next week

            future = client.map(derive_fb_test_error, npa_rmse_inner)        

            rmses_inner = []

            for f in as_completed(future):
                if(f.status=="error"):
                    rmses_inner.append(np.nan)
                else:
                    rmses_inner.append(f.result())

            rmses_inner_ = [t[2] for t in rmses_inner]
            rmse_ = [r for r in rmses_inner_]

            rmse_inner_flags = [np.nanmean(rmse_),np.nanstd(rmse_)]

            print("inner cv scores:")
            inner_rmse_df = pd.DataFrame([rmse_inner_flags],columns=['mean','std'])

            scores = inner_rmse_df['mean'].values
            sdevs = inner_rmse_df['std'].values
            print("best inner cv score, sdev (11 folds):",scores[np.argmin(scores)],sdevs[np.argmin(scores)])

            future = client.map(derive_fb_test_error, npa_rmse_outer)
            results_rmses_outer = []

            for f in as_completed(future):
                if(f.status=="error"):
                    results_rmses_outer.append([np.nan])
                else:
                    results_rmses_outer.append(f.result()) 

            results_rmses_outer_ = [t[2] for t in results_rmses_outer]
            test_score_mean = np.nanmean([r for r in results_rmses_outer_])
            test_score_std = np.nanstd([r for r in results_rmses_outer_])

            client = Client('192.168.3.100:8786')

            y_s = [t[0] for t in results_rmses_outer]
            y_hats = [t[1] for t in results_rmses_outer]
            test_errors_ = np.array(y_s)-np.array(y_hats)
            np.std(test_errors_,axis=0)
            std_resid = test_errors_/((np.sum(test_errors_)**2)/(len(test_errors_)-2)**.5)

            std_error_test = np.std(test_errors_,axis=0)

            print("test cv mean error, stdev (10 folds):",test_score_mean,test_score_std)

            m = Prophet(interval_width=0.95)

            prophet_df = data['Adj Close'].reset_index()#data.reset_index()
            prophet_df.columns = ['ds','y']
            prophet_df['y'] = np.log(prophet_df['y'])
            m.fit(prophet_df)

            future_ = m.make_future_dataframe(periods = n_ahead,freq='W-'+nyse_dates.index[-1].strftime('%a'))

            forecast = m.predict(future_).set_index('ds')[['yhat','yhat_lower','yhat_upper']]
            df_pred = np.exp(forecast)

            #fb forecast
            temp_df = pd.DataFrame(data['Adj Close']).reset_index()
            temp_df.columns = ['ds','Adj Close']
            temp_df.set_index('ds',inplace=True)

            s_date = df_pred.tail(n_ahead).index[np.argmax(df_pred['yhat'].tail(n_ahead).values)].strftime('%Y-%m-%d')

            print("sell date:",s_date)
            e_return = np.array((np.max(df_pred['yhat'].tail(n_ahead).values)-metrics_df_['Adj Close'])/metrics_df_['Adj Close'])[0]
            p_metrics = pd.DataFrame(df_pred.tail(n_ahead).iloc[np.argmax(df_pred['yhat'].tail(n_ahead).values)]).T
            print("expected return:",e_return)

            days_delta = (datetime.strptime(s_date, '%Y-%m-%d') - datetime.strptime(end.strftime('%Y-%m-%d'), '%Y-%m-%d')).days
            try:
                discounted_return = (1+e_return)**(1/days_delta)-1
            except:
                discounted_return = 0
            qtr_return = (1+discounted_return)**92

            print("discounted return:",discounted_return)
            print("quarter return:",qtr_return)

            #95% 2 tail
            t_score = stats.t.ppf(1-0.025, len(test_errors_))

            prediction_interval_yhat_upper = df_pred[['yhat']].tail(n_ahead)+(std_error_test.reshape(n_ahead,1)*t_score)
            prediction_interval_yhat_lower = df_pred[['yhat']].tail(n_ahead)-(std_error_test.reshape(n_ahead,1)*t_score)

            cv_outter_std_ratio = test_score_std / e_return
            cv_outter_err_ratio = test_score_mean / e_return                        

            stop_loss_price = pd.DataFrame(np.mean(pd.concat([df_pred[['yhat']].tail(n_ahead),prediction_interval_yhat_lower],axis=1),axis=1)).iloc[0].values[0]
            decision_metrics_df_ = pd.DataFrame([s,e_return,discounted_return,qtr_return,s_date,stop_loss_price,days_delta,discounted_return,qtr_return,test_score_mean,test_score_std,mean_revert_flag,p_metrics['yhat_upper'].values[0],p_metrics['yhat_lower'].values[0]])#,columns=)#,index=[s_])#.sort_values(by='disc_rtn',ascending=False,inplace=True)
            decision_metrics_df_ = decision_metrics_df_.T
            decision_metrics_df_.columns=['SYMBOL','exp_return','discounted_return','qtr_return','sell_date','stop_loss_price','days_delta','disc_rtn','qtr_rtn','outer_cv_avg_abs_err_n11','outer_cv_std_n11','mean_revert_flag','yhat_upper','yhat_lower']
            decision_metrics_df_.index = [symbol_]       

            bbands_ = bbands[set(bbands.columns).difference(['Adj Close'])]
            bbands_.index = [symbol_]

            outer_cv_avg_abs_err_n11_ratio = pd.DataFrame(test_score_mean/screener_sorted.loc[symbol_]['Adj Close'],columns=['outer_cv_avg_abs_err_n11_ratio'],index=[symbol_])
            outer_cv_std_n11_ratio = pd.DataFrame(test_score_std/screener_sorted.loc[symbol_]['Adj Close'],columns=['outer_cv_std_n11_ratio'],index=[symbol_])


            decision_metrics_df_ = pd.concat([decision_metrics_df_,bbands_,metrics_df_[metrics_df_.columns.difference(['21dBOLD', '21dMA-TP', '21dBOLU','90dSMA', '90dSDev'])],outer_cv_avg_abs_err_n11_ratio,outer_cv_std_n11_ratio],axis=1)
            #decision_metrics_df_ = decision_metrics_df_.sort_values(by='exp_return',ascending=True)                        

            print("Outer CV Error / Price Ratio",decision_metrics_df_['outer_cv_avg_abs_err_n11_ratio'])
            print(" Outer CV StdDev / Price Ratio",decision_metrics_df_['outer_cv_std_n11_ratio'])

            decision_metrics_df = pd.concat([decision_metrics_df,decision_metrics_df_],axis=0)            

            savePlot=False
            if(decision_metrics_df_['outer_cv_avg_abs_err_n11_ratio'].values[0]<.9):
                if(decision_metrics_df_['outer_cv_std_n11_ratio'].values[0]<.9):
                    if(decision_metrics_df_['qtr_return'].values[0]>1.089):
                        savePlot=True                 

            plt.plot(temp_df)
            plt.plot(df_pred)
            plt.legend(['Adj Close','yhat','yhat_lower','yhat_upper'],loc=2)
            plt.xticks(rotation = 45,size=8)

            if(savePlot):
                plt.savefig(end.strftime('%Y-%m-%d')+'_'+symbol_+'_full_forecast.png', dpi=300, format='png', bbox_inches='tight')           
            plt.show()

            plt.plot(temp_df.tail(n_ahead))
            plt.plot(df_pred.tail(n_ahead*2))
            plt.plot(prediction_interval_yhat_upper,linestyle = 'dashed')
            plt.plot(prediction_interval_yhat_lower,linestyle = 'dashed')

            plt.legend(['Adj Close','yhat','yhat_lower','yhat_upper','pi_upper','pi_lower'],loc=2)

            plt.xticks(rotation = 45,size=8)

            if(savePlot):
                plt.savefig(end.strftime('%Y-%m-%d')+'_'+symbol_+'_13w_forecast.png', dpi=300, format='png', bbox_inches='tight')
            plt.show()

            fcst = m.predict(future_)
            fig = m.plot(fcst)

            if(savePlot):
                plt.savefig(end.strftime('%Y-%m-%d')+'_'+symbol_+'_fb_sim_forecast.png', dpi=300, format='png', bbox_inches='tight')
            plt.show()            

            if(np.sum(np.array(fundamental_entries)==s)>0):
                loc = np.where(np.array(fundamental_entries)==s)[0][0]
                for item in fundamentals_quarterlies[loc][1]:
                    if(type(item[1]) == type(None)):
                        pass
                    else:
                        if(len(item[1])>0):
                            #print(len(item[1]))
                            print(item)
            client.close()


    decision_metrics_df.sort_values(by='exp_return',ascending=False,inplace=True)

    display(decision_metrics_df)

    V=v.Experiment()
    V.start()
    V.ds = decision_metrics_df
    V.fe = decision_metrics_df
    V.end()
    decision_metrics_df.to_csv(end.strftime('%Y-%m-%d')+'_decision_metrics_df.csv')

    counts_df = pd.DataFrame(decision_metrics_df['Sector'].value_counts())

    print(counts_df)
    fig, ax = plt.subplots()
    ax.pie(counts_df['Sector'], labels=counts_df.index)
    ax.set_title('Sector Participartion > Risk_Trend_Factor of 0')
    plt.tight_layout()
    plt.savefig(end.strftime('%Y-%m-%d')+'_piechart-sectors.png', dpi=300, format='png', bbox_inches='tight')

In [ ]:
def colored(r, g, b, text):
    return f"\033[38;2;{r};{g};{b}m{text}\033[0m"

In [ ]:
def display_side_by_side(dfs:list, captions:list, tablespacing=5):
    """Display tables side by side to save vertical space
    Input:
        dfs: list of pandas.DataFrame
        captions: list of table captions
    """
    output = ""
    for (caption, df) in zip(captions, dfs):
        output += df.style.set_table_attributes("style='display:inline'").set_caption(caption)._repr_html_()
        output += tablespacing * "\xa0"
    display(HTML(output))

In [ ]:
cm = sns.color_palette("blend:red,yellow,green", as_cmap=True)

def b_g(s, cmap=cm, low=0, high=0):
    # Pass the columns from Dataframe A 
    a = A.loc[:,s.name].copy()
    #rng = a.max() - a.min()
    rng = np.nanmax(A.values.ravel()) - np.nanmin(A.values.ravel())
    norm = mp.colors.Normalize(np.nanmin(A.values.ravel()) - (rng * low),
                        np.nanmax(A.values.ravel()) + (rng * high))
    normed = norm(a.values)
    #c = [mp.colors.rgb2hex(x) for x in plt.cm.get_cmap(cmap)(normed)]
    c = [mp.colors.rgb2hex(x) for x in plt.cm.get_cmap(cm)(normed)]
    return ['background-color: %s' % color for color in c]

In [ ]:
nyse = mcal.get_calendar('NYSE')
nyse_dates = nyse.schedule(start_date=start, end_date=end)

In [ ]:
sector_performance_return = pd.DataFrame()
sector_performance_Q_smoothed_return = pd.DataFrame()

for v in [*indexes['Symbol'],*sectors['Symbol']]:
    subset = prices_df[prices_df['Symbol'] == v]
    #subset_ = subset.set_index('Date').asfreq('Q')[['Adj Close']].pct_change()
    subset_ = subset.asfreq('D').interpolate().asfreq('Q')[['Adj Close']].pct_change()
    #print(subset_)
    #subset_qs = subset.set_index('Date')[['Adj Close']].pct_change().rolling(63).mean()
    subset_qs = subset[['Adj Close']].pct_change().rolling(63).mean()
    subset_.index = subset_.index.values.astype('M8[D]')
    subset_qs.index = subset_qs.index.values.astype('M8[D]')
    dt_str = subset_.index[-1].strftime('%Y-%m-%d')
    #tail = subset[subset.set_index('Date').index>=dt_str]
    tail = subset[subset.index>=dt_str]
    #dt_str = subset.set_index('Date').index[-1].strftime('%Y-%m-%d')
    dt_str = subset.index[-1].strftime('%Y-%m-%d')
    #tail_ = pd.DataFrame([(tail.set_index('Date')['Adj Close'][-1]-tail.set_index('Date')['Adj Close'][0])/tail.set_index('Date')['Adj Close'][0]],index=[datetime.strptime(dt_str, '%Y-%m-%d')],columns=['Adj Close'])
    tail_ = pd.DataFrame([(tail['Adj Close'][-1]-tail['Adj Close'][0])/tail['Adj Close'][0]],index=[datetime.strptime(dt_str, '%Y-%m-%d')],columns=['Adj Close'])
    subset__ = pd.concat([subset_,tail_],axis=0)
    sector_performance_return = pd.concat([sector_performance_return,subset__],axis=1)
    sector_performance_Q_smoothed_return = pd.concat([sector_performance_Q_smoothed_return,subset_qs],axis=1)

In [ ]:
sector_performance_Q_smoothed_return.columns = [*[i for i in indexes.index],*[v for v in sectors.index]]
sector_performance_return.columns = [*[i for i in indexes.index],*[v for v in sectors.index]]

In [ ]:
"""
from scipy import stats as st
from scipy.stats import t

std_indexes = pd.DataFrame()
seasonal_indexes = pd.DataFrame()

warnings.filterwarnings('ignore')
for spr in sector_performance_return.columns:
    #print(spr)
    s = sector_performance_return[[spr]]
    #print(s)
    years = []
    quarters = []
    for d in sector_performance_return[spr].index:

        d_ = d.strftime('%Y-%m-%d')
        #print(d_)

        datem = dt.datetime.strptime(d_, "%Y-%m-%d")
        years.append(str(datem.year))
        #print(datem.year)       # 2021
        m = datem.month
        d = datem.day
        if(len(str(m))==1):
            m = '0'+str(m)
        else:
            m = str(m)
        if(len(str(d))==1):
            d = '0'+str(d)
        else:
            d = str(d)
        quarters.append(m+"-"+d)

    s['Years'] = years
    s['Quarters'] = quarters
    ct = pd.crosstab(s['Years'], s['Quarters'],values=s[spr],aggfunc=np.mean).mean(axis=0).sort_index()
    ct.columns = [spr]
    
    ct_std = pd.crosstab(s['Years'], s['Quarters'],values=s[spr],aggfunc=np.mean).std(axis=0).sort_index()
    ct_std.columns = [spr]
    
    std_indexes = pd.concat([std_indexes,ct_std],axis=1)
    seasonal_indexes = pd.concat([seasonal_indexes,ct],axis=1)

seasonal_indexes.columns = sector_performance_return.columns

std_indexes.columns = sector_performance_return.columns
    
seasonal_indexes = seasonal_indexes.T

std_indexes = std_indexes.T

si = ['03-31','06-30','09-30','12-31']

print("Seasonal Indexes")

seasonal_index_values_sorted = []
seasonal_index_names_sorted = []
#B = seasonal_indexes[si]
for c in seasonal_indexes[si].columns:
    seasonal_index_values_sorted.append(seasonal_indexes[si][c].sort_values(ascending=False).values)
    seasonal_index_names_sorted.append(seasonal_indexes[si][c].sort_values(ascending=False).index.values)

temp_seasonal_index_values_sorted = pd.DataFrame(seasonal_index_values_sorted).T
temp_seasonal_index_values_sorted.columns = seasonal_indexes[si].columns

temp_seasonal_index_names_sorted = pd.DataFrame(seasonal_index_names_sorted).T
temp_seasonal_index_names_sorted.columns = seasonal_indexes[si].columns

A = temp_seasonal_index_values_sorted
B = temp_seasonal_index_names_sorted 

display(B.style.apply(b_g,cmap=cm))

display(seasonal_indexes[si].style.background_gradient(cmap = cm,axis=0))

print("Note if the final data in the prices ends on a quarter, this will aggregate into the seasonal_index above and there will be no comparison")
print("Current")

display(seasonal_indexes[seasonal_indexes.columns.difference(si)].sort_values(by=seasonal_indexes[seasonal_indexes.columns.difference(si)].columns[0],ascending=False).style.background_gradient(cmap = cm,axis=0))    

#calculate p-value
n=5

print("standard deviations")
display(std_indexes[si])
t_scores = (seasonal_indexes[si])/(std_indexes[si]/np.sqrt(n))
print("t scores")
display(t_scores)
print("p values")
p_values = pd.DataFrame(t.cdf(t_scores, df=n-2),index=seasonal_indexes[si].index,columns=seasonal_indexes[si].columns)

display(p_values.style.applymap(highlight_cells, color_if_true='green', color_if_false='yellow', threshold_=.5))

p_values.hist()
"""

In [ ]:
cycles_ = ['Expansion','Slowdown','Recession','Recovery']

path=r"C:\Users\User\Documents\wiki\wiki\dev\python\Python-Stock\reports\figures\Business cycles.png"
display(Image.open(path))
path=r"C:\Users\User\Documents\wiki\wiki\Finance\Business Cycles-2.png"
display(Image.open(path))

In [ ]:
sector_performance_returned_no_dup = sector_performance_return.loc[~sector_performance_return.index.duplicated(keep ='first')]
sector_performance_return_pvt = sector_performance_returned_no_dup[sector_performance_returned_no_dup.iloc[-1].sort_values(ascending=False).index]

sector_performance_return_pvt_heatmap = sector_performance_return_pvt.style.background_gradient(cmap = cm,axis=None)
display(sector_performance_return_pvt_heatmap)

sector_performances = pd.DataFrame()
sector_performances_values = pd.DataFrame()
for c in sector_performance_returned_no_dup.T.columns:
    #print(c)
    temp = pd.DataFrame(sector_performance_returned_no_dup.T[c].sort_values(ascending=False).index,columns=[c])
    #print(temp)
    temp2 = pd.DataFrame(sector_performance_returned_no_dup.T[c].sort_values(ascending=False),columns=[c])
    #print(temp2)
    temp2.reset_index(drop=True,inplace=True)    
    sector_performances = pd.concat([sector_performances,temp],axis=1)
    sector_performances_values = pd.concat([sector_performances_values,temp2],axis=1)
    
A = sector_performances_values
B = sector_performance_returned_no_dup.T

df = B.style.apply(b_g,cmap='RdYlGn')

display(df)


In [ ]:
"""
#cm = sns.color_palette("blend:red,yellow,green", as_cmap=True)
#pd.crosstab(shares_viz['Year'], shares_viz['Month'])

#seasonal_indexes[si]
seasonally_adjusted = pd.DataFrame()

for s in sector_performance_return.columns:
    
    s_ = pd.DataFrame(sector_performance_return[s])
    #print(sector_performance_return[s])
    years = []
    quarters = []
    
    for d in s_.index:
        d_ = d.strftime('%Y-%m-%d')
        #print(d_)

        datem = dt.datetime.strptime(d_, "%Y-%m-%d")
        years.append(str(datem.year))
        #print(datem.year)       # 2021
        m = datem.month
        d = datem.day
        if(len(str(m))==1):
            m = '0'+str(m)
        else:
            m = str(m)
        if(len(str(d))==1):
            d = '0'+str(d)
        else:
            d = str(d)
        quarters.append(m+"-"+d)
        
    s_['Quarter'] = quarters
    si_ = seasonal_indexes.loc[s]
    
    modified = pd.DataFrame()
    #print(si_.index)
    for s_i_ in si_.index:
        #print(s_i_)
        #when it finds the last date (which hopefully isn't a quarter date)
        if(np.sum(np.array(si)==s_i_)==0):
            #print("don't modify")
            #print(s_i_)
            match = []
            for entry in si:
                if(int(s_i_.rsplit("-")[0])<=int(entry.rsplit("-")[0])):
                    match = entry
                    break            

            delta_months = int(match.rsplit("-")[0])-int(s_i_.rsplit("-")[0])
            delta_days_ = int(match.rsplit("-")[1])-int(s_i_.rsplit("-")[1])
            
            delta_days = int(delta_months*(91.25/3)+delta_days_)
            #daily interest
            
            #print(delta_days)
            #print(seasonal_indexes.loc[s].loc[match])
            seasonal_index_to_daily = (1+seasonal_indexes.loc[s].loc[match])**(1/91.25)-1
            #print("Seasonal Index to Daily Discounted Interest Rate:",seasonal_index_to_daily)
            daily_extrapolated = (1+seasonal_index_to_daily)**delta_days-1
            #print(daily_extrapolated)
            #print("Daily ^ delta_days:",daily_extrapolated)
            
            #print(match)
            #print(s_i_)
            seasonally_adjusted_column = pd.DataFrame([seasonal_indexes.loc[s].loc[match] - daily_extrapolated],columns=[s],index=[s_.index[-1]])
            
            
        else:
            seasonally_adjusted_column = pd.DataFrame(s_[s][s_['Quarter']==s_i_] - seasonal_indexes.loc[s].loc[s_i_],columns=[s])
        modified = pd.concat([modified,seasonally_adjusted_column],axis=0)
        modified.sort_index(inplace=True)
    #print(modified)
    seasonally_adjusted = pd.concat([seasonally_adjusted,modified],axis=1)

print("Comparative to Seasonal Indexes, tells you who is over or underperforming.")
print("This is Seasonal Index - Extrapolated Current Quarter.")
print("+: seasonal > current")
print("-: current > seasonal")

print("Note if the final data in the prices ends on a quarter, this will aggregate into the seasonal_index above and there will be no comparison")
    
    
std_indexes_seas_adj = pd.DataFrame()
seasonal_indexes_seas_adj = pd.DataFrame()

temp_ = pd.DataFrame(seasonally_adjusted.T.iloc[:,-1])
temp_.columns = pd.DataFrame(seasonally_adjusted.T.iloc[:,-1]).columns.strftime('%Y-%m-%d')
display(temp_.sort_values(by=temp_.columns[0],ascending=False).style.background_gradient(cmap = cm,axis=0))

#display(seasonally_adjusted[seasonally_adjusted.columns.difference(si)].style.background_gradient(cmap = cm,axis=0))

for spr in seasonally_adjusted.columns:
    #print(spr)
    s = seasonally_adjusted[[spr]]
    #print(s)
    years = []
    quarters = []
    for d in seasonally_adjusted[spr].index:
        d_ = d.strftime('%Y-%m-%d')
        #print(d_)

        datem = dt.datetime.strptime(d_, "%Y-%m-%d")
        years.append(str(datem.year))
        #print(datem.year)       # 2021
        m = datem.month
        d = datem.day
        if(len(str(m))==1):
            m = '0'+str(m)
        else:
            m = str(m)
        if(len(str(d))==1):
            d = '0'+str(d)
        else:
            d = str(d)
        quarters.append(m+"-"+d)

    s['Years'] = years
    s['Quarters'] = quarters
    ct = pd.crosstab(s['Years'], s['Quarters'],values=s[spr],aggfunc=np.mean).mean(axis=0).sort_index()
    ct.columns = [spr]
    #print(ct)
    
    ct_std = pd.crosstab(s['Years'], s['Quarters'],values=s[spr],aggfunc=np.mean).std(axis=0).sort_index()
    ct_std.columns = [spr]
    
    std_indexes_seas_adj = pd.concat([std_indexes_seas_adj,ct_std],axis=1)
    seasonal_indexes_seas_adj = pd.concat([seasonal_indexes_seas_adj,ct],axis=1)

seasonal_indexes_seas_adj.columns = seasonally_adjusted.columns

std_indexes_seas_adj.columns = seasonally_adjusted.columns
    
seasonal_indexes_seas_adj = seasonal_indexes_seas_adj.T

std_indexes_seas_adj = std_indexes_seas_adj.T

sector_performances_sea_adj = pd.DataFrame()
sector_performances_values_sea_adj = pd.DataFrame()
for c in seasonally_adjusted.T.columns:
    temp = pd.DataFrame(seasonally_adjusted.T[c].sort_values(ascending=False).index,columns=[c])
    temp2 = pd.DataFrame(seasonally_adjusted.T[c].sort_values(ascending=False),columns=[c])
    temp2.reset_index(drop=True,inplace=True)    
    sector_performances_sea_adj = pd.concat([sector_performances_sea_adj,temp],axis=1)
    sector_performances_values_sea_adj = pd.concat([sector_performances_values_sea_adj,temp2],axis=1)

A = sector_performances_values_sea_adj
B = sector_performances_sea_adj

df_sea_adj = B.style.apply(b_g,cmap='RdYlGn')
display(df_sea_adj)
"""

In [ ]:
"""
index_performance = []
for v in indexes['Symbol'].values:
    index_performance.append([v,screener.loc[v]['risk_trend_factor']])
temp_i = pd.DataFrame(index_performance,index=indexes.index)
temp_i.columns = ['Symbol','risk_trend_factor']
index_risk_sorted = pd.DataFrame(index_performance,columns=['Symbol','risk_trend_factor'],index=indexes.index).sort_values(by='risk_trend_factor',ascending=False).reset_index()
index_risk_sorted.columns = ['Index','Symbol','risk_trend_factor']
index_risk_sorted
"""

In [ ]:
"""
sector_performance = []
for v in sectors['Symbol'].values:
    sector_performance.append([v,screener.loc[v]['risk_trend_factor']])
temp_v = sectors.reset_index()
temp_v.columns = ['Sector','Symbol']
sectors_risk_sorted = pd.DataFrame(sector_performance,columns=['Symbol','risk_trend_factor']).merge(temp_v, on='Symbol', how='left').sort_values(by='risk_trend_factor',ascending=False)    
sectors_risk_sorted
"""

In [ ]:
print("Rolling Quarterly Mean Return")
plt.plot(sector_performance_Q_smoothed_return)
plt.legend([*[i for i in indexes.index],*[v for v in sectors.index]],loc=2,fontsize=7)
plt.show()

In [ ]:
print ("Quarterly Returns")
for v in sector_performance_return.T.index:
    subset = pd.DataFrame(sector_performance_return.T.loc[v])
    subset.columns = ['Return']
    plt.plot(subset)
    plt.xticks(subset.index, rotation=45)
plt.legend(sector_performance_return.T.index,loc=2)    
plt.show()


In [ ]:
"""
plt.plot(seasonal_indexes[si].T)
plt.legend(seasonal_indexes[si].T.columns,loc=2,fontsize=8)
plt.show()
"""

In [ ]:
#threshold = 0

In [ ]:
#risk_trend_threshold

In [ ]:
#market_data['sector'] = stock_fundamentals['sector']

In [ ]:
#screener[['risk_trend_factor']].replace(['missing','error'], np.NaN).dropna().hist()

In [ ]:
#len(screener)

In [ ]:
#prices_df.columns

In [ ]:
#threshold

In [ ]:
#threshold_value = summary.dropna()['risk_trend_factor'].quantile([threshold])

In [ ]:
fred_pvt_sample = completed_fred_pvt_df[fred_names].asfreq('D').interpolate(method='time').asfreq('Q').dropna()

In [ ]:
temp_filter = screener_sorted['risk_trend_factor'].replace(['missing','error'], np.nan).dropna()

In [126]:
filtered_screener_sorted = screener_sorted.loc[temp_filter[temp_filter>0].index]

In [127]:
filtered_screener_sorted

,Date,MIN_Lookback_One_Year,MIN_Lookback_2_Months,MIN_Lookback_1_Months,risk_trend_factor,risk_per_share,volume_factor,Adj Close,21dBOLD,21dMA-TP,21dBOLU,90dSMA,90dSDev,adf,hurst,sector_risk_trend_factor,Sector,Sector Symbol,Market Index,Index Symbol
ACC,2022-07-12 00:00:00,49.900002,65.019997,65.220001,10.333332,1.380001,1.678518,64.800003,64.049767,64.550755,65.051743,64.224127,1.744527,0.97795,0.430192,error,error,error,SP400,MDYG
PSB,2022-07-12 00:00:00,165.850006,189.830002,189.830002,5.812073,2.819992,2.202007,187.300003,185.670469,186.713214,187.755959,184.428572,6.66043,0.82777,0.481505,error,error,error,SP400,MDYG
MTOR,2022-07-12 00:00:00,32.93,36.279999,36.299999,5.694921,0.59,1.604974,36.290001,35.923105,36.170833,36.418561,36.059048,0.166188,0.343949,0.438584,error,error,error,SP600,SLY
Y,2022-07-12 00:00:00,737.890015,862.869995,862.869995,3.903303,9.099976,1.697274,833.049988,829.852572,832.469957,835.087342,834.369563,3.638737,0.463779,0.464478,-5.291467,Financial Services,VFH,SP400,MDYG
TVTY,2022-06-30 00:00:00,27.870001,32.889999,32.889999,3.195021,1.205,1.881193,32.5,31.852152,32.262421,32.672689,32.181865,0.232209,0.525779,0.444727,error,error,error,SP600,SLY
MANT,2022-07-12 00:00:00,92.0,92.0,95.980003,3.018012,2.220001,1.951703,95.559998,94.445493,95.207698,95.969903,90.537222,6.117749,0.526043,0.367929,error,error,error,SP600,SLY
HRB,2022-07-12 00:00:00,26.23,30.48,36.830002,2.926828,5.740002,0.776919,36.779999,32.408951,35.035794,37.662636,32.068849,4.06568,0.927917,0.448481,-8.127184,Consumer Cyclical,VCR,SP400,MDYG
MCK,2022-07-12 00:00:00,204.660004,339.940002,339.940002,2.431491,40.139984,1.185482,327.380005,298.235041,317.350318,336.465595,320.842213,8.540538,0.998807,0.460534,-2.728791,Healthcare,VHT,SP500,SPY
FCN,2022-07-12 00:00:00,147.380005,170.309998,172.690002,2.265068,30.859985,1.018294,186.759995,156.31092,174.765913,193.220906,167.920913,8.208045,0.976035,0.400169,-5.230956,Industrials,VIS,SP400,MDYG
GCP,2022-07-12 00:00:00,27.780001,32.279999,32.279999,2.246695,1.135,2.089532,31.629999,30.743752,31.290575,31.837399,31.175476,0.252875,0.177273,0.45789,-3.192567,Basic Materials,VAW,SP600,SLY


In [ ]:

with localconverter(ro.default_converter + pandas2ri.converter):
      r_from_pd_df = ro.conversion.py2rpy(fred_pvt_sample.melt(ignore_index=False).reset_index().rename(columns={'index': 'Date'}))


In [ ]:
#ro.X11()
#ro.windows()
ro.r('''

my_mean <- function(x, na.rm=TRUE) {
  mean(x, na.rm = na.rm)
}

f <- function(y) {
#print(y)

#library(arfima)
#varvefd = arfima(y)
#d = summary(varvefd)$coef[[1]][1]
#return(d)

tsfeature_tbl <- y %>%
group_by(variable) %>%
tk_tsfeatures(
  .date_var = Date,
  .value    = value,
  .period   = 4,
  .features = c("frequency", "stl_features", "entropy", "acf_features", "my_mean"),
  .scale    = TRUE,
  .prefix   = "ts_"
) %>%
ungroup()
    
print(tsfeature_tbl)

set.seed(123)

cluster_tbl <- tibble(
    cluster = tsfeature_tbl %>% 
        select(-variable) %>%
        as.matrix() %>%
        kmeans(centers = 7, nstart = 100) %>%
        pluck("cluster")
) %>%
    bind_cols(
        tsfeature_tbl
    )

cluster_tbl

cluster_tbl %>%
    select(cluster, variable) %>%
    right_join(y, by = "variable") %>%
    group_by(variable) %>%
    plot_time_series(
      Date, value, 
      .color_var   = cluster, 
      .facet_ncol  = 2, 
      .interactive = FALSE
    )
plot(cluster_tbl)

return(tsfeature_tbl)
}
''')
grdevices.png(file="temp.png", width=4096, height=1024)
r_f = ro.globalenv['f']
d=(r_f(r_from_pd_df))
#rprint(pp)

time.sleep(3)
#grdevices.dev_copy(device = r.png, filename = "plot.png", width = 1000, height = 500)
grdevices.dev_off()

#From here optional, if you want a waiting time
#Elsewise close the plot manually afterwards with grdevices.dev_off()

#grdevices.dev_off()
#grdevices.dev_off()


In [ ]:
pca = PCA(n_components=0.99, svd_solver='full')

X = np.array(d.iloc[:,1:])

pca.fit(scale(X))
#pca.explained_variance_
pca.explained_variance_ratio_.cumsum()
X_pca = pd.DataFrame(pca.transform(d.iloc[0:,1:]))
X_pca.index = d.index
X_pca.sort_values(by=[0],ascending=False,inplace=True)

TSS_ = []
BSS_ = []
WSS_ = []
silhouettes_ = []

for k in range(2,int(len(d)/2)):
    model = KMeans(n_clusters=k, random_state=0, n_init=100).fit(X)
    #print(model.inertia_)

    codebook = np.array(model.cluster_centers_)
    partition, euc_distance_to_centroids = vq(X, codebook)
    WSS = np.sum(euc_distance_to_centroids**2)
    
    silhouette_avg = silhouette_score(X, model.labels_)

    silhouettes_.append(silhouette_avg)
    
    TSS = np.sum((X-X.mean(0))**2)

    BSS = TSS - WSS

    TSS_.append(TSS)
    BSS_.append(BSS)
    WSS_.append(WSS)
    
    #print(TSS, WSS, BSS)


In [ ]:

init_min = 2
init_max = 5


mink = 2
maxk = 6

tss, bss, wss = findOptimalK_ANOVA(X_pca, mink = mink, maxk = maxk)

In [ ]:
plt.plot(np.array(BSS_)/np.array(TSS_))
plt.show()
plt.plot(np.array(WSS_))
plt.show()
plt.plot(np.array(silhouettes_))
plt.show()



In [ ]:

bss_ = findknee(bss)
bss_ = bss_/np.max(bss_)

wss_ = findknee(np.array(pd.DataFrame(wss).mean(1)))
wss_ = wss_/np.max(wss_)

temp_df = pd.DataFrame(bss_/wss_).replace([np.inf, -np.inf, np.NaN], 0)
temp_df.index = np.array(range(0,len(wss_)))+mink
#plt.plot(temp_df)

set_ = np.abs(temp_df-1)
plt.plot(set_)

optimal_k = np.argmin(set_)+mink
plt.plot(abs(set_-1))
 
print(optimal_k)


In [ ]:
clf = KMeansConstrained(n_clusters=optimal_k, size_min=init_min, size_max=max(np.ceil(len(X_pca)/optimal_k),init_max), init='k-means++', n_init=100, max_iter=100, tol=0.0001, verbose=False, random_state=None, copy_x=True, n_jobs=4)
clf.fit_predict(X_pca)

labels = clf.labels_
clusters = clf.n_clusters
centers = clf.cluster_centers_

print(labels)

tot_ss, BSS, within_ss = deriveANOVA(clf, X_pca)

Global_F = (BSS/(optimal_k-1))/(np.mean(within_ss)/(len(X_pca)-optimal_k))
global_sig = 1-f.cdf(Global_F, (len(X_pca)-optimal_k), len(X_pca)-1)

ind_F_scores = []

for w in range(0,len(within_ss)):

    dfn = (optimal_k-1)
    dfd = np.sum(labels==w)-optimal_k

    F_score = (BSS/dfn)/(within_ss[w]/(dfd))
    ind_F_scores.append(F_score)

print("Global F:",Global_F)
print("Global Sig:",global_sig)
print("F-Scores:", ind_F_scores)

print("P-Scores:", 1-f.cdf(ind_F_scores, dfn, dfd))

X_pca['cluster'] = labels

In [ ]:


cgram = Clustergram(range(1, 8))
cgram.fit(X_pca)
cgram.plot()
cgram

In [ ]:

lengths = []
for i in range(0,max(X_pca['cluster'])+1):
    subset = d.loc[X_pca[X_pca['cluster']==i].index]
    lengths.append(len(subset))
    
colors = ['purple','cyan','magenta','green','red','black','pink','yellow','blue','brown','orange','grey']
my_cmap = LinearSegmentedColormap.from_list(
    'color_map', colors, N=max(lengths))

rescale = lambda y: (y - 0) / (np.max(lengths) - 0)    


In [ ]:

for i in range(0,max(X_pca['cluster'])+1):
    print("Group: ",i)
    subset = d.loc[X_pca[X_pca['cluster']==i].index]
    #
    for c_ in range(0,len(subset['variable'])):
        c = subset['variable'].values[c_]
        try:
            #print(dict_fred[c])
            display(html_print(' '.join([cstr(ti, color=ci) for ti,ci in ((dict_fred[c], colors[c_]),)])))

            #display(html_print(' '.join([cstr(ti, color=my_cmap(rescale(c_))) for ti in (('hello my name is'),)])))
            #print(colored("hello red world", my_cmap(rescale(c_))))
            if(c_==0):
                fig, ax1 = plt.subplots( figsize=(30,4))
                ax1.plot(fred_pvt_sample[c],color=my_cmap(rescale(c_)))
                ax1.get_yaxis().set_ticks([])
            else: 
                ax2 = ax1.twinx()
                ax2.plot(fred_pvt_sample[c],color=my_cmap(rescale(c_)))
                ax2.get_yaxis().set_ticks([])
        #ax1.legend(subset['variable'].values,loc=2)
        except:
            pass
    plt.show()

In [ ]:
filter_ = fred_pvt_sample.columns.tolist()
def f3(Y):
    
    #Y = x
    #output_slider_variable.value
    internalFilter = filter_.copy()
    internalFilter.remove(Y)
    all_data_ = pd.concat([fred_pvt_sample[Y],fred_pvt_sample[internalFilter]], axis=1)    
    #print(all_data_.describe())
    display(fred_pvt_sample.describe())
    #x_ticks = all_data_.index[np.arange(0, len(all_data.index), int(len(internalFilter)/5))]
    x_ticks  = []
    for index, element in enumerate(fred_pvt_sample.index):
        if index % int(np.round(len(fred_pvt_sample.index)/10)) == 0:
            x_ticks.append(element)
    plt.plot(fred_pvt_sample[Y])
    plt.xticks(x_ticks, rotation = 45)
    plt.show()        
    plt.hist(fred_pvt_sample[Y], bins='auto')
    plt.show()
    diff = pd.DataFrame((fred_pvt_sample[Y].pct_change())).dropna()
    plt.hist(diff, bins='auto')
    plt.show()
    return(fred_pvt_sample)
    
out = interactive(f3, Y=filter_)

#output_slider_variable.observe(f4, 'value')

print("choose Y")
display(out)


In [ ]:
print('nominal,','pct_change,','pct_change().cumsum')
for pos in range(0,len(fred_names)):
    print(fred_friendly_names[pos],fred_names[pos])
    name = fred_names[pos]
    
    
    f, (ax1, ax2, ax3) = plt.subplots(1, 3, sharey=False,figsize=(15,6))
    ax1.plot(completed_fred_pvt_df[name])
    ax1.set_xticklabels(np.array(completed_fred_pvt_df.index.map(lambda t: t.strftime('%Y-%m-%d')))[np.arange(0,len(completed_fred_pvt_df.index),int(len(completed_fred_pvt_df.index)/10))], rotation=45)
    ax2.plot(completed_fred_pvt_df[name].pct_change())
    ax2.set_xticklabels(np.array(completed_fred_pvt_df.index.map(lambda t: t.strftime('%Y-%m-%d')))[np.arange(0,len(completed_fred_pvt_df.index),int(len(completed_fred_pvt_df.index)/10))], rotation=45)
    ax3.plot(completed_fred_pvt_df[name].pct_change().cumsum())
    ax3.set_xticklabels(np.array(completed_fred_pvt_df.index.map(lambda t: t.strftime('%Y-%m-%d')))[np.arange(0,len(completed_fred_pvt_df.index),int(len(completed_fred_pvt_df.index)/10))], rotation=45)
    plt.show()

In [ ]:
#screener['vol_30d_2yr']

In [ ]:
#pd.DataFrame(np.mean(np.exp(final_model.get_prediction(start=end,end=(end + dt.timedelta(92)).strftime('%Y-%m-%d')).summary_frame())[['mean','pi_lower']],axis=1)).iloc[0].values[0]
#[s,e_return,s_date,stop_loss_price,days_delta,discounted_return,qtr_return,test_score_mean,test_score_std,mean_revert_flag,p_metrics['pi_lower'].values[0],p_metrics['pi_upper'].values[0]]
#screener['Adj Close'].loc[decision_metrics_df.index]

In [ ]:
mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False

#tscv = expanding_window(initial =52*2, horizon = 13,period = 26)

#batchClearLimit = round(len(list__)/7)

print("adf < .05 or .01, mean reverting")
print("Hurst")
print("> .5 - The time series is mean reverting.")
print("= .5 - The time series is a Geometric Brownian Motion.")
print("< .5 - The time series is trending.")
"""*indexes['Symbol'].values,*sectors['Symbol'].values,"""
"""
widgets.Dropdown(
    #options=stocks_,
    #value=None,
    description='Choose Stock:',
    disabled=False,
)

#y_=widgets.Select(options=stocks_,disabled=False)
y_=widgets.Select(options=np.sort([*list_sector_n_indexes,*list_stocks]),value=None,disabled=False)

a=interact(plot_,symbol_=y_)
"""

In [129]:
def process_values(values):
    
    sector_list = []
    symbol_list = []
    for i in np.array(values):
        t= dict_sectors[i]
        sector_list.append(t)
        
        sublist = filtered_screener_sorted[filtered_screener_sorted['Sector Symbol']==t].index
        temp_df = filtered_screener_sorted.loc[sublist]
        #temp_df_ = temp_df['risk_trend_factor'].replace(['error','missing'], np.nan).dropna()
        
        #temp_df_filtered = temp_df_[temp_df_>0]
        
        flat_list = [item for item in temp_df.index]
        symbol_list.extend(flat_list)        
    
    button = widgets.Button(description="Process")
    output = widgets.Output()

    display(button, output)

    print("Symbols > 0 trend_factor_score")
    print(len(sector_list),sector_list)
    print(len(symbol_list),symbol_list)
    
    def on_button_clicked(b):
        with output:            
            plot_(symbol_list)

    button.on_click(on_button_clicked)    
    
y_=widgets.SelectMultiple(
    options=list(dict_sectors.keys()),
    description='Choose Sectors: ',
    disabled=False
)

a=interact(process_values,values=y_)

interactive(children=(SelectMultiple(description='Choose Sectors: ', options=('Basic Materials', 'Communicatio…